## HRA Hierarchical Tissue Unit Annotation

In this notebook, we will build on [an existing one on hierarchical tissue unit annotation](https://github.com/HickeyLab/Hierarchical-Tissue-Unit-Annotation) by [Dr. John Hickey](https://bme.duke.edu/people/john-hickey/). Outputs from this workbook are featured in Fig. 3 in the paper titled "Human BioMolecular Atlas Program (HuBMAP): 3D Human Reference Atlas Construction and Usage" (accepted for publication in Nature Methods, see preprint [on bioRxiv](https://doi.org/10.1101/2024.03.27.587041)). In this notebook, we will create a Vasculature Common Coordinate Framework (VCCF) distance visualization for the same donor and region (B006, proximal jejunum) as in the paper, see below.

<img src="images/fig_3_original.jpg" width="400">


Concretely, we will take the original CSV file from Dryad with cell positions, types, donor IDs, and extraction sites, and then create a node-dist-vis widget plus a Cell Distance Explorer widget. For more information and documentation on hra-jupyter-widgets, please see [https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb](https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb).

## Load libraries

In [1]:
# Import native packages
import os
from pprint import pprint
import csv

In [ ]:
#Install and import external packages
%pip install matplotlib pandas ipywidgets hra_jupyter_widgets

import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets

# Import hra-jupyter-widgets. For documentation, please see https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb
from hra_jupyter_widgets import (
    BodyUi,
    CdeVisualization, # in this example, we will use this one...
    Eui,
    EuiOrganInformation,
    FtuExplorer,
    FtuExplorerSmall,
    MedicalIllustration,
    ModelViewer,
    NodeDistVis, # ...and this one, but all of them are usable for different purposes!
    Rui,
)

## Download data from Dryad

Let's get the data from Dryad. Note that this is a ~3GB file. We will use `curl` to dowload it.

In [ ]:
# Make sure the data folder is present
folder_path = "data"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

# Define the path to the file. 
file_path = f'{folder_path}/23_09_CODEX_HuBMAP_alldata_Dryad_merged.csv'

# Check if the file exists
if not os.path.exists(file_path):
    # If the file doesn't exist, run the curl command
    !curl -L https://datadryad.org/api/v2/files/2572152/download -o {file_path}
    print(f"File downloaded and saved at {file_path}")
else:
    print(f"File already exists at {file_path}")

## Read data as DataFrame

In [ ]:
# Read the CSV file and convert it to a df
df = pd.read_csv(file_path, index_col=0)
df

In [5]:
# Only keep cells from one dataset by selecting 1 donor and 1 region
df_filtered = df[(df['donor'] == "B012") & (
    df['unique_region'] == "B012_Proximal jejunum")]

In [6]:
df_filtered['Cell Type'] = df_filtered['Cell Type'].astype(str) + '_Cell Type'
df_filtered['Neighborhood'] = df_filtered['Neighborhood'].astype(str) + '_Neighborhood'
df_filtered['Community'] = df_filtered['Community'].astype(str) + '_Community'
df_filtered['Tissue Unit'] = df_filtered['Tissue Unit'].astype(str) + '_Tissue Unit'

In [ ]:
df_filtered

In [8]:
# Next, let's define a function that turns a DataFrame into a node list that can then be passed into the CdeVisualization or NodeDistVis widget
def make_node_list(df:pd.DataFrame, is_3d:bool = False):
  """Turn a DataFrame into a list of dicts for passing them into a HRA widget

  Args:
      df (pd.DataFrame): A DataFrame with cells
  """
  
  # If the df does not have a z-axis column, let's add one and set all cells to 0
  if not is_3d:
    df.loc[:, ('z')] = 0
  
  node_list = [{'x': row['x'], 'y': row['y'], 'z': row['z'], 'level': row['combined_info']}
                 for index, row in df.iterrows()]

  return node_list
  

In [9]:
# # Prepare df_cells for visualization with NodeDistVis widget
# node_list = make_node_list(df_cells, False)

# # Let's inspect the first 5 rows
# pprint(node_list[:5])

# Nodes and edges test

In [ ]:
'''
import csv

# Define the path to the CSV file
csv_file_path = '/Users/yang/Desktop/3D_visualization/data/CRC01002-nodes.csv'

# Initialize an empty list to store the nodes
node_list = []

# Open the CSV file
with open(csv_file_path, mode='r') as file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(file)
    
    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Append each row (as a dictionary) to the node_list
        node_list.append(row)
'''

In [11]:
# node_list

In [ ]:
'''
import csv

# Define the path to the CSV file
csv_file_path = '/Users/yang/Desktop/3D_visualization/data/CRC01002-edges.csv'

# Initialize an empty list to store the nodes
edge_list = []

# Open the CSV file
with open(csv_file_path, mode='r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Append each row (as a dictionary) to the node_list
        edge_list.append(row)
'''

In [13]:
# edge_list

In [ ]:
'''
node_dist_vis = NodeDistVis(
    nodes=node_list,
    node_target_key="Cell Type",
    node_target_value="Endothelial",
    edges=edge_list,
)
display(node_dist_vis)
'''

# Display

In [15]:
# # Finally, let's instantiate the NodeDistVis class with some parameters. We pass in the node_list, indicate Endothelial cells as targets for the edges. 
# # As we are not supplying an edge list, we need to provide a max_edge_distance, which is set to 1000 (generiously)
# node_dist_vis = NodeDistVis(
#     nodes = node_list,
#     node_target_key="Cell Type",
#     node_target_value="Endothelial",
#     max_edge_distance = 1000
# )

# # Display our new widget
# display(node_dist_vis)

For comparison, here is Fig. 3 from the paper for comparison with the `NodeDistVis` widget above. Note that this figure shows **cell neighborhoods** rather than **cell types** like the `NodeDistVis`:

<img src="images/fig_3_original.jpg" width="400">

## Next, let's make four regions and visualize a 3D tissue stack.

To showcase the 3D rendering capabilities of our `CDEVisualization` widget, we will display the same section as we showed with the `NodeDistVis` widget above, but we will show it three times with an added offset along the z-axis.

In [ ]:
df_filtered

In [17]:
# Create a new data frame with values from the NodeDistVis example
df = df_filtered

# indicate the number of layers you would like to show. In our case, let's show 3.
n_layers = 4   # One layer for cell type, one for neighborhood, one for community, one for tissue unit

# Create a list to hold all the data frames
df_list = [df]

for i in range(0, n_layers-1):

  # Create a copy of this new DataFrame
  df_copy = df.copy()

  # Modify a column in the copied rows (e.g., change values in column 'B')
  df_copy['unique_region'] = f'copy_{i}'  

  # Add df_copy to list of df
  df_list.append(df_copy)
  
# Concatenate the original DataFrame with the modified copies
df_combined = pd.concat(df_list, ignore_index=True)

df_filtered_3d = df_combined

In [18]:
# Set a z-offset
offset = 2500

# Set z axis (or any other axis) by region
df_filtered_3d['z'] = df_filtered_3d['unique_region'].apply(lambda v: 0 if v == 'B012_Proximal jejunum'
                                                            else offset if v == 'copy_0'
                                                            else offset * 2 if v == 'copy_1'
                                                            else offset * 3
                                                            )

# Make new df with only x, y, z, Cell Type, and Neighborhood columns
df_cells_3d = df_filtered_3d[['x', 'y', 'z', 'Cell Type', 'Neighborhood', 'Community', 'Tissue Unit']]

cell_type_z0 = df_cells_3d[df_cells_3d['z'] == 0]['Cell Type']
neighborhood_z1 = df_cells_3d[df_cells_3d['z'] == offset]['Neighborhood']
community_z2 = df_cells_3d[df_cells_3d['z'] == offset *2]['Community']
tissueunit_z3 = df_cells_3d[df_cells_3d['z'] == offset *3]['Tissue Unit']

combined_info = pd.concat([cell_type_z0, neighborhood_z1, community_z2, tissueunit_z3])

df_cells_3d['combined_info'] = combined_info


In [ ]:
len(df_cells_3d['combined_info'].unique())

In [ ]:
df_cells_3d

In [21]:
df_cells_3d_filtered = df_cells_3d[['x', 'y', 'z', 'combined_info']]

In [22]:
# Prepare df_cells_3d for visualization with CdeVisualization widget
node_list = make_node_list(df_cells_3d_filtered, True)

# # Let's inspect the first 5 rows
# pprint(node_list[:5])

# for n in node_list:
#     print(n['level'])

# Nodes and edges list construction

In [23]:
# Filter the DataFrame where the content in the 'combined_info' column is the level name interested
edge_df = df_cells_3d_filtered[df_cells_3d_filtered['combined_info'] == 'Plasma Cell Enriched_Neighborhood']

# Select the 'x', 'y', and 'z' columns from the filtered DataFrame
edge_df_filtered = edge_df[['x', 'y', 'z', 'combined_info']]


In [24]:
# Extract the corresponding information in the cell types
edge_index = edge_df_filtered.index - len(df_filtered)
edge_df_2 = df_cells_3d_filtered.iloc[edge_index]

In [25]:
edge_df_filtered = edge_df[['x', 'y', 'z']]
edge_df_2_filtered = edge_df_2[['x', 'y', 'z']]
edge_df_filtered = edge_df_filtered.reset_index(drop=True)
edge_df_2_filtered = edge_df_2_filtered.reset_index(drop=True)

In [26]:
# Combine the selected columns horizontally
edge_list_df = pd.concat([edge_df_filtered, edge_df_2_filtered], axis=1)

edge_list_df = edge_list_df.reset_index(drop=False)

# Convert the DataFrame to a list of lists with each value as a string
edge_list = edge_list_df.astype(str).values.tolist()

In [ ]:
edge_list

# Display

In [ ]:
# Finally, let's instantiate the CDEVisualization class with our node_list as parameter.
cde = CdeVisualization(
    nodes=node_list,
    node_target_key = "level",
    #node_target_value = "Enterocyte_Cell Type",
    edges=edge_list,
)

# Display our new widget
display(cde)